# Levenberg-Marquardt mothod for ordinary differential equations

The purpose is to produce
1. $\varphi(\mathbf{z})$ that produces $n\times(p+1)$-dimension vector with input arguments of function, initial condition $\mathbf{y}_0$, and parameters $\mathbf{k}$.
2. Levenberg-Marquardt parameter estimation procedure.

In [4]:
import numpy as np
from numpy import linalg as LA
from scipy.integrate import solve_ivp, odeint

## ODE model and objective function
For the intial value problem
\begin{equation}
\frac{\text{d}\mathbf{y}(t)}{\text{d}t}=\mathbf{f}(\mathbf{y}(t),\mathbf{k});\hspace{10mm}\mathbf{y}(t_0)=\mathbf{y}_0
\end{equation}
where $\mathbf{k}$ is a parameter vector with $p$ elements, the objective function with the measurements $\hat{\mathbf{y}}_i$ ($i=1,\cdots,N$) and the weighting matrix $\mathbf{Q}_i$ is
\begin{equation}
S(\mathbf{k})=\frac{1}{2}\sum_{i=1}^N\left[\hat{\mathbf{y}}_i-\mathbf{y}(t_i,\mathbf{k})\right]^\top\mathbf{Q}_i\left[\hat{\mathbf{y}}_i-\mathbf{y}(t_i,\mathbf{k})\right]=\frac{1}{2}\sum_{i=1}^N\mathbf{r}_i(\mathbf{k})^\top\mathbf{Q}_i\mathbf{r}_i(\mathbf{k})
\end{equation}
where $S(\mathbf{k}):\mathbb{R}^p\to\mathbb{R}$ and $\mathbf{r}_i(\mathbf{k})=\hat{\mathbf{y}}_i-\mathbf{y}(t_i,\mathbf{k})$ is the residuals. The gradient of the objective function is
\begin{equation}
\mathbf{g}=\frac{\partial S(\mathbf{k})}{\partial\mathbf{k}}=-\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i
\end{equation}
and the Hessian is
\begin{align}
\mathbf{H}&=\frac{\partial^2S(\mathbf{k})}{\partial\mathbf{k}^2}=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i-\sum_{i=1}^N\frac{\partial\mathbf{J^\top}_i}{\partial\mathbf{k}}\mathbf{Q}_i\mathbf{r}_i\\
&\simeq\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i
\end{align}
where $\mathbf{J}_i(\mathbf{k})=\frac{\partial\mathbf{y}(t_i,\mathbf{k})}{\partial\mathbf{k}}$ is the Jacobian.

## Talyor expansion of scalar function of several variables
The line passes $\mathbf{x}_0$ along the direction $\mathbf{s}$ is the set of points satisfying $\mathbf{x}(\alpha)=\mathbf{x}_0+\alpha\mathbf{s}$ for all $\alpha$. For a scalar value function $f(\mathbf{x})$, the directional derivative of $f$ at $\mathbf{x}_0$ in the direction of $\mathbf{s}$ is,
\begin{equation}
D_\mathbf{s}f(\mathbf{x}_0)=\lim_{\alpha\to0}\frac{f(\mathbf{x}_0+\alpha\mathbf{s})-f(\mathbf{x}_0)}{\alpha}=\frac{\text{d}f(\textbf{x})}{\text{d}\alpha}\bigg|_{\textbf{x}=\textbf{x}_0}
\end{equation}
By the chain rule,
\begin{equation}
\frac{\text{d}}{\text{d}\alpha}=\sum_i\frac{\partial}{\partial x_i}\frac{\text{d}x_i(\alpha)}{\text{d}\alpha}=\sum_is_i\frac{\partial}{\partial x_i}=\mathbf{s^\top \nabla}
\end{equation}
so the slope of $f$ along any line $\mathbf{x}(\alpha)$ is
\begin{equation}
\frac{\text{d}f}{\text{d}\alpha}=\mathbf{s}^\top\nabla f=\nabla f^\top\mathbf{s}
\end{equation}
and the curvature along the line is
\begin{equation}
\frac{\text{d}^2 f}{\text{d}\alpha^2}=\frac{\text{d}}{\text{d}\alpha}\frac{\text{d}f}{\text{d}\alpha}=\mathbf{s}^\top\nabla(\nabla f^\top\mathbf{s})=\mathbf{s}^\top\nabla^2f\mathbf{s}.
\end{equation}.
The Taylor expansion of multiple variables is
\begin{equation}
f(\mathbf{x}_0+\alpha\mathbf{s})=f(\mathbf{x}_0)+\alpha\mathbf{s}^\top\nabla f(\mathbf{x}_0)+\frac{1}{2}\alpha^2\mathbf{s}^\top\left[\nabla^2f(\mathbf{x}_0)\right]\mathbf{s}+\cdots
\end{equation}
or
\begin{equation}
f(\mathbf{x}_0+\mathbf{h})=f(\mathbf{x}_0)+\mathbf{h}^\top\nabla f(\mathbf{x}_0)+\frac{1}{2}\mathbf{h}^\top\left[\nabla^2f(\mathbf{x}_0)\right]\mathbf{h}+\cdots.
\end{equation}

## Gauss-Newton method
The Taylor expansion of $\mathbf{r}_i$ respect to $\mathbf{h}$ is
$$\mathbf{r}_i(\mathbf{k+h})=\mathbf{r}_i(\mathbf{k})+\frac{\partial\mathbf{r}_i(\mathbf{k})}{\partial\mathbf{k}}\mathbf{h}+\mathcal{O}(\mathbf{h^\top h})=\mathbf{r}_i(\mathbf{k})-\mathbf{J}_i(\mathbf{k})\mathbf{h}+\mathcal{O}(\mathbf{h^\top h}).$$  
In Gauss-Newton (GN) method, $\mathbf{r}_i$ is approximated to a linear model
\begin{equation}
\mathbf{l}_i=\mathbf{r}_i(\mathbf{k})-\mathbf{J}_i(\mathbf{k})\mathbf{h}
\end{equation}
Inserting $\mathbf{l}_i$ to the objective function yields
\begin{align}
S(\mathbf{k+h})&\simeq L(\mathbf{h})=\frac{1}{2}\sum_{i=1}^N\mathbf{l}_i^\top(\mathbf{h})\mathbf{Q}_i\mathbf{l}_i(\mathbf{h})\\
    &=\frac{1}{2}\sum_{i=1}^N\left[\mathbf{r}_i\mathbf{(k)}^\top\mathbf{Q}_i\mathbf{r}_i\mathbf{(k)}-\mathbf{r}_i^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{(k)h}-\mathbf{h}^\top\mathbf{J}_i\mathbf{(k)}^\top\mathbf{Q}_i\mathbf{r}_i(\mathbf{k})+\mathbf{h}^\top\mathbf{J}_i\mathbf{(k)}^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{(k)}\mathbf{h}\right]\\
    &=S(\mathbf{k})-\sum_{i=1}^N\mathbf{h^\top J}_i^\top\mathbf{Q}_i \mathbf{r}_i+\frac{1}{2}\sum_{i=1}^N\mathbf{h^\top J}_i^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{h}
\end{align}
since $\mathbf{r}_i^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{(k)h}=\mathbf{h}^\top\mathbf{J}_i\mathbf{(k)}^\top\mathbf{Q}_i\mathbf{r}_i(\mathbf{k})$.  
The gradient and Hessian of $L$ are
\begin{equation}
\mathbf{L'(h)}=\sum_{i=1}^N\left[-\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i+\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i\mathbf{h}\right]\hspace{20mm}\mathbf{L''(h)}=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i
\end{equation}
The Hessian is independent of $\mathbf{h}$, symmetric and positive definite if $\mathbf{J}$ has full rank. Hence $L$ is minimum when $\mathbf{L'(h)}$ is zero vector. The step $\mathbf{h}$ can be calculated by,
\begin{equation}
\left[\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i\right]\mathbf{h}=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\left(\hat{\mathbf{y}}_i-\mathbf{y}(t_i,\mathbf{k})\right)
\end{equation}
so that
\begin{equation}
\mathbf{Hh=-g}
\end{equation}
GN method is not quadratic convergent if $\mathbf{r}_i$ is not zero around the solution.


## Levenberg-Marquardt method
In Levenberg-Marquardt (LM) method, GN method is used with a damping term,
\begin{equation*}
\left[\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{J}_i+\mu\mathbf{I}\right]\mathbf{h}_\text{lm}=\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i
\end{equation*}
For large $\mu$, $\mathbf{h}_\text{lm}\simeq-\frac{1}{\mu}\mathbf{L'(0)}$ is a short step in the steepest descent direction. This is good if the estimation is far from the solution. If $\mu$ is very small, LM method is nearly GN method which is almost quadratic convergent if $S(\mathbf{k})$ is close to zero.
### Initial $\mu$
The initial value of $\mu$ is maximum diagonal element of $\mathbf{H}_0=\sum\mathbf{J}_i(\mathbf{k}_0)^\top\mathbf{Q}_i\mathbf{J}_i(\mathbf{k}_0)$,
\begin{equation*}
\mu_0=\tau\cdot\max[\text{diagonal}(\mathbf{H}_0)]
\end{equation*}
where $\tau$ is small such as $10^{-6}$ if $\mathbf{k}_0$ is believed to be a good approximation or large such as $10^{-3}$ or $1$ otherwise.
### Gain ratio
The updating of $\mu$ is controlled by the gain ratio
\begin{equation*}
\rho=\frac{S(\mathbf{k})-S(\mathbf{k+h}_\text{lm})}{L(\mathbf{0})-L(\mathbf{h}_\text{lm})}
\end{equation*}
The denominator is the gain predicted by the linear model,
\begin{align*}
L(\mathbf{0})-L(\mathbf{h}_\text{lm})=&\mathbf{h}_\text{lm}^\top\sum_{i=1}^N\mathbf{J}_i\mathbf{Q}_i\mathbf{r}_i-\frac{1}{2}\mathbf{h}^\top_\text{lm}\left[\sum_{i=1}^N\mathbf{J}_i\top\mathbf{Q}_i\mathbf{J}_i\right]\mathbf{h}_\text{lm}\\
                   =&\frac{1}{2}\mathbf{h}_\text{lm}^\top\left(2\sum_{i=1}^N\mathbf{J}_i\mathbf{Q}_i\mathbf{r}_i-\left[\sum_{i=1}^N\mathbf{J}^\top_i\mathbf{Q}_i\mathbf{J}_i+\mu\mathbf{I}\right]\mathbf{h}_\text{lm}+\mu\mathbf{I}\mathbf{h}_\text{lm}\right)\\
                   =&\frac{1}{2}\mathbf{h}_\text{lm}^\top\left(\sum_{i=1}^N\mathbf{J}_i^\top\mathbf{Q}_i\mathbf{r}_i+\mu\mathbf{h}_\text{lm}\right)\\
                   =&\frac{1}{2}\mathbf{h}_\text{lm}^\top\left(-\mathbf{g}+\mu\mathbf{h}_\text{lm}\right)
\end{align*}
A large value of $\rho$ indicates that $L(\mathbf{h}_\text{lm})$ is a good approximation of $S(\mathbf{k+h}_\text{lm})$ so $\mu$ can be decreased so that LM is closer to GN method.



## Calculation of Jacobian in ODE models
In ODE models, the sensitivity matrix cannot be obtained by a simple differentiation. Instead, we can get differential equation for $\mathbf{J}$.
Differentiate both side of $ \frac{\text{d}\mathbf{y}}{\text{d}t}=\mathbf{f}(\mathbf{y}(t),\mathbf{k})$ and apply the chain rule,
\begin{align}
\frac{\partial}{\partial\mathbf{k}}\left(\frac{\text{d}\mathbf{y}}{\text{d}t}\right)=\frac{\text{d}}{\text{d}t}\left(\frac{\partial\mathbf{y}}{\partial\mathbf{k}}\right)&=\frac{\partial}{\partial\mathbf{k}}\mathbf{f}(\mathbf{y}(t),\mathbf{k})\\
&=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\frac{\partial\mathbf{y}}{\partial\mathbf{k}}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}\frac{\partial\mathbf{k}}{\partial\mathbf{k}}\\
&=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\frac{\partial\mathbf{y}}{\partial\mathbf{k}}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}
\end{align}
Hence,
\begin{equation*}
\frac{\text{d}\mathbf{J}(t)}{\text{d}t}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{J}(t)+\frac{\partial\mathbf{f}}{\partial\mathbf{k}};\hspace{10mm}\mathbf{J}(t_0)=0
\end{equation*}


## Contruction of ODE system with Jacobian
The Jacobian or the sensitivity matrix is
\begin{equation}
\mathbf{J}(t)=\frac{\partial\mathbf{y}}{\partial\mathbf{k}}=\left[\frac{\partial\mathbf{y}}{\partial k_1},\cdots,\frac{\partial\mathbf{y}}{\partial k_p}\right]=[\mathbf{g}_1,\cdots,\mathbf{g}_p]
\end{equation}
where $\mathbf{g}_j$ represents $n$-dimensional vector which is the sensitivity coefficients of the state variables with respect to parameter $k_j$. Each of $\mathbf{g}_j$ satisfies the differential equation for Jacobian such that
\begin{equation*}
\frac{\text{d}\mathbf{g}_j(t)}{\text{d}t}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_j+\frac{\partial\mathbf{f}}{\partial k_j};\hspace{10mm}\mathbf{g}_j(t_0)=0;\hspace{10mm}j=1,\cdots,p
\end{equation*}
We generate $n\times(p+1)$-dimensional differential equations system
\begin{equation*}
\frac{d\mathbf{z}}{dt}=\varphi(\mathbf{z})
\end{equation*}
$\mathbf{z}$ is $n\times(p+1)$-dimensional vector
\begin{equation*}
\mathbf{z}=\begin{bmatrix} \mathbf{x}(t)\\
                          \frac{\partial\mathbf{y}}{\partial k_1}\\
                          \vdots\\
                          \frac{\partial\mathbf{y}}{\partial k_p}
\end{bmatrix}
=\begin{bmatrix} \mathbf{y}(t)\\
                 \mathbf{g}_1(t)\\
                 \vdots\\
                 \mathbf{g}_p(t)
\end{bmatrix}
\end{equation*}
$\mathbf{\varphi}(\mathbf{z})$ is $n\times(p+1)$-dimensional vector function

\begin{equation*}
\mathbf{\varphi}(\mathbf{z})=\begin{bmatrix}
\mathbf{f}(\mathbf{y},\mathbf{k})\\
\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_1(t)+\frac{\partial\mathbf{f}}{\partial k_1}\\
\vdots\\
\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_p(t)+\frac{\partial\mathbf{f}}{\partial k_p}
\end{bmatrix}
\end{equation*}

To get the Jacobian for all $t_i$, $\varphi(\mathbf{z}_i)$ should be solved for $t_i,~~i=1,2,\cdots,N$.


### Arrange of the solution
The Jacobian $\textbf{J}$ is obtained by integration of $\varphi(z)$. Integration of $\varphi(z)$ returns $n\times(p+1)$ vector
\begin{equation*}
\textbf{z}=\begin{bmatrix}
\textbf{y}\\
\textbf{g}_1\\
\textbf{g}_2\\
\vdots\\
\textbf{g}_p
\end{bmatrix}
\end{equation*}
where
\begin{equation*}
\textbf{g}_j=\begin{bmatrix}
\frac{\partial y_1}{\partial k_j}\\
\frac{\partial y_2}{\partial k_j}\\
\vdots\\
\frac{\partial y_n}{\partial k_j}
\end{bmatrix},\hspace{20mm}j=1,\cdots,p
\end{equation*}
To compute the Hessian
\begin{equation*}
\mathbf{H}=\sum_{i=1}^N\mathbf{J}(t_i)^\top\mathbf{Q}_i\mathbf{J}(t_i)
\end{equation*}
the Jacobian for all measurement time $\mathbf{J}(t_1),\cdots,\mathbf{J}(t_N)$, should be returned.
The ODE solver for initial value problem returns $[n\times(p+1)]\times N$ matrix
\begin{equation*}
Z=\begin{bmatrix}
\mathbf{y}(t_1)&\mathbf{y}(t_2)&\cdots&\mathbf{y}(t_N)\\
\mathbf{g}_1(t_1)&\mathbf{g}_1(t_2)&\cdots&\mathbf{g}_1(t_N)\\
\vdots&&\ddots&\vdots\\
\mathbf{g}_p(t_1)&\mathbf{g}_p(t_2)&\cdots&\mathbf{g}_p(t_N)
\end{bmatrix}
\end{equation*}
This matrix would be refomulated for
\begin{equation*}
\textbf{Y}=\begin{bmatrix}
\mathbf{y}(t_1)&\mathbf{y}(t_2)&\cdots&\mathbf{y}(t_N)
\end{bmatrix}
\end{equation*}
and
\begin{align*}
\mathcal{J}=&\begin{bmatrix}
\mathbf{J}(t_1),\mathbf{J}(t_2),\cdots,\mathbf{J}(t_N)
\end{bmatrix}\\
=&\left[\begin{bmatrix}
\textbf{g}_1(t_1)&\textbf{g}_2(t_1)&\cdots&\textbf{g}_p(t_1)
\end{bmatrix}
,\cdots,\begin{bmatrix}
\textbf{g}_1(t_N)&\textbf{g}_2(t_N)&\cdots&\textbf{g}_p(t_N)
\end{bmatrix}\right].
\end{align*}
Note that $\mathbf{Y}$ is an $n\times N$ matrix and $\mathcal{J}$ is an $n\times p\times N$ tensor.

In [11]:
def integrator_jacobian(ode,y0,k,time):
    n = np.size(y0)
    p = np.size(k)
    N = np.size(time)
    # initial condition J0 = 0
    z0 = np.zeros(n*(p+1))
    z0[0:n] = y0
    def dzdt(t,z):
        return phi_ode(ode,z,k,n,p)
    solution = solve_ivp(dzdt,[time[0],time[-1]],z0,method='Radau',t_eval=time)
    if solution.success == False:
        raise OverflowError("Integration by integrator_jacobian failed")
    Z = solution.y
    Y = Z[0:n]
    J = Z[n:]
    Jt = np.hsplit(J,N)
    for i in range(N):
        Jt[i] = Jt[i].reshape(p,n).transpose()
    return Y,Jt

def phi_ode(ode,z,k,n,p):
    y = z[0:n]
    J = z[n:].reshape((p,n)).transpose()
    phiz = np.empty(n*(p+1))
    dfdy = dfdy_ode(ode,y,k,n)
    dfdk = dfdk_ode(ode,y,k,n,p)
    dJdt = dfdy@J+dfdk
    phiz[0:n] = ode(y,k)
    phiz[n:] = dJdt.transpose().flatten()
    return phiz

def dfdy_ode(ode,y,k,n):
    h = 1e-8
    y = y.astype(np.float)
    if np.isscalar(y):
        dfdy = (ode(y+h,k)-ode(y-h,k))/(2*h)
        return dfdy
    else:
        dfdy = np.empty((n,n))
        for i in range(n):
            yr = y.copy()
            yl = y.copy()
            yr[i] += h
            yl[i] -= h
            dfdy[i] = (ode(yr,k)-ode(yl,k))/(2*h)
        return dfdy.transpose()
    return

def dfdk_ode(ode,y,k,n,p):
    h = 1e-8
    if p == 1:
        dfdk = (ode(y,k+h)-ode(y,k-h))/(2*h)
        return dfdk.reshape(n,1)
    else:
        k = k.astype(np.float)
        dfdk = np.empty((p,n))
        for i in range(p):
            kr = k.copy()
            kl = k.copy()
            kr[i] += h
            kl[i] -= h
            dfdk[i] = (ode(y,kr)-ode(y,kl))/(2*h)
        return dfdk.transpose()
    return

### Reduced Jacobian
The Hessian $\mathbf{H}$ might be ill-conditioned if the paramters differ by several order of magnitude. To overcome numerical problem, reduced Jacobian $\mathbf{J}_\text{R}$ is defined as
$$\mathbf{J}_\text{R}=\mathbf{JK}$$
where $\mathbf{K}=\text{diag}(k_1,\cdots,k_p)$. Then we can solve
$$\mathbf{H}_\text{R}\mathbf{h}_\text{R}=-\mathbf{g}_\text{R}$$
where $\mathbf{H}_\text{R}=\mathbf{KHK}$ and $\mathbf{g}_\text{R}=\mathbf{Kg}$.
We can restore $\textbf{h}$ from $\textbf{h=Kh}_\text{R}$

## Algorithm of LM method




In [1]:
def lm(ode,yhat,Q,k0,time,opts=[1e-3,1e-8,1e-8,1000]):
    # Input arguments

    # opts = [tau, tolg, tolk, max_iter]
    #
    # Outputs
    # output = [k,Y,info]
    # k : parameters
    # Y : results with k
    # info = [it,ter]
    # it : Number of iterations
    # ter : Termination criteria 1: gradient 2: change in h 3: maximum iteration

    try:
        stop = False
        nu = 2
        it = 0  
        rho = 0
        ter = 'm'
        if np.size(yhat) == np.size(yhat,0):
            y0 = yhat[0]
            N = np.size(yhat)
        else:
            y0 = yhat[:,0]
            N = np.size(yhat,1)
        p = np.size(k0)
        I = np.eye(p)
        k = k0
        print('Iter | Obj func | red grad | lin appr |   mu   |   rho')
        Y, J = integrator_jacobian(ode,y0,k,time)
        S, r = objective_func(yhat,Y,Q,N)
        S0 = S
        H,g = Hg(J,Q,r,p,N)
        K = np.diag(k)
        Hr = K@H@K
        gr = K@g
        gn = LA.norm(gr,np.inf)
        stop = bool(gn < opts[1])
        if stop:
            ter = 'g'
        mu = opts[0]*max(np.diag(Hr))
        print("{0:5d}|{1:10.4e}|{2:10.2e}|  Not cal |{3:8.1e}| Not cal"
              .format(it,S,gn,mu))
        while (not stop) and (it<=opts[3]):
            it += 1
            hr = svdsolve(Hr+mu*I,-gr)
            dphi = np.dot(hr,gr)
            print(dphi)
            h = K@hr
            hn = LA.norm(h,np.inf)
            kn = LA.norm(k,np.inf)
            if hn <= opts[2]*(kn+opts[2]):
                stop = True
                ter = 'h'
            else:
                k_new = k + h
                Y, J = integrator_jacobian(ode,y0,k_new,time)
                S, r = objective_func(yhat,Y,Q,N)
                lin = h.T@(mu*h-g)/2
                rho = (S0 - S)/lin
                if rho >0:
                    k = k_new 
                    K = np.diag(k)
                    S0 = S
                    H, g = Hg(J,Q,r,p,N)
                    Hr = K@H@K
                    gr = K@g
                    gn = LA.norm(gr,np.inf) 
                    if gn < opts[1]:
                        stop = True
                        ter = 'g'
                    mu *= max(1/3,1-(2*rho-1)**3)
                    nu = 2
                else:
                    mu *= nu
                    nu *= 2
            if rho == 0:
                print("{0:5d}|{1:10.4e}|{2:10.2e}|{3:10.2e}|{4:8.1e}| Not calculated"
                      .format(it,S,gn,lin,mu))
            else:
                print("{0:5d}|{1:10.4e}|{2:10.2e}|{3:10.2e}|{4:8.1e}|{5:8.1e}"
                      .format(it,S,gn,lin,mu,rho))
        info = [it,ter]
        output = [k,Y,info]
        return output
    except OverflowError:
        print("Problem with integration. Try with another parameter")
        return

def objective_func(yhat,Y,Q,N):
    S = 0
    r = yhat-Y
    if np.size(Q) == 1:
        S = np.sum(r**2)/2
    else:
        for i in range(N):
            # S += np.dot(np.matmul(r[:,i],Q),r[:,i])
            S += r[:,i]@Q@r[:,i]/2
    return S, r

def Hg(J,Q,r,p,N):
    H = np.zeros((p,p))
    g = np.zeros(p)
    for i in range(N):
        JQ = J[i].T@Q
        H += JQ@J[i]
        g -= JQ@r[:,i]
    return H,g

def svdsolve(A,b):
    u,s,v = np.linalg.svd(A)
    c = u.T@b
    w = np.linalg.solve(np.diag(s),c)
    x = v.T@w
    return x